```{contents}
```
## Configuration Management

Configuration management in **LangGraph** controls **how a graph executes at runtime without changing its structure**.
It separates **workflow logic** from **execution behavior**, enabling safe experimentation, scaling, observability, and production governance.

---

### **1. Why Configuration Management Matters**

Without configuration, every change requires **rewriting the graph**.
With configuration, you can:

| Goal                    | Achieved By    |
| ----------------------- | -------------- |
| Switch models           | Runtime config |
| Change recursion limits | Runtime config |
| Control retries         | Runtime config |
| Inject callbacks        | Runtime config |
| Enable tracing          | Runtime config |
| Tune performance        | Runtime config |

> **Design principle:**
> *Graph = algorithm, Config = operating policy*

---

### **2. Core Components of LangGraph Configuration**

LangGraph uses a structured **runtime configuration dictionary**:

```python
config = {
    "configurable": {...},
    "recursion_limit": 25,
    "callbacks": [...],
    "tags": [...],
    "metadata": {...}
}
```

| Section         | Purpose                     |
| --------------- | --------------------------- |
| configurable    | Custom runtime parameters   |
| recursion_limit | Maximum loop steps          |
| callbacks       | Tracing, logging, streaming |
| tags            | Label executions            |
| metadata        | Attach structured info      |

---

### **3. Configurable Fields**

Nodes can declare **configurable parameters**:

```python
from langchain_core.runnables import ConfigurableField

model = ChatOpenAI().configurable_fields(
    temperature=ConfigurableField(id="temperature"),
    model_name=ConfigurableField(id="model")
)
```

Runtime injection:

```python
graph.invoke(input, config={
    "configurable": {
        "temperature": 0.2,
        "model": "gpt-4o"
    }
})
```

**Effect:** same graph, different behavior.

---

### **4. Controlling Cycles & Safety**

```python
graph.invoke(input, config={
    "recursion_limit": 20
})
```

| Setting         | Role                    |
| --------------- | ----------------------- |
| recursion_limit | Prevent infinite loops  |
| timeout         | Prevent hung executions |
| max_concurrency | Control parallelism     |

---

### **5. Observability & Debugging**

```python
from langchain.callbacks import StdOutCallbackHandler

graph.invoke(input, config={
    "callbacks": [StdOutCallbackHandler()],
    "tags": ["experiment-A"],
    "metadata": {"user_id": "42"}
})
```

This enables:

* Execution tracing
* Token usage tracking
* Error localization
* Performance profiling

---

### **6. Environment-Based Configuration**

Typical production pattern:

```python
import os

config = {
    "configurable": {
        "model": os.getenv("LLM_MODEL"),
        "temperature": float(os.getenv("TEMP"))
    }
}
```

| Environment | Behavior                  |
| ----------- | ------------------------- |
| Dev         | High logging, low limits  |
| Staging     | Medium limits             |
| Prod        | Strict limits, monitoring |

---

### **7. Configuration in Multi-Agent Systems**

Each agent can receive **independent configuration**:

```python
graph.invoke(input, config={
    "configurable": {
        "planner_model": "gpt-4",
        "executor_model": "gpt-3.5-turbo"
    }
})
```

---

### **8. Production Configuration Checklist**

| Layer         | Configuration                  |
| ------------- | ------------------------------ |
| LLM           | model, temperature, max_tokens |
| Safety        | recursion_limit, timeout       |
| Scale         | max_concurrency                |
| Observability | callbacks, tags                |
| Cost          | token limits                   |
| Governance    | metadata, audit tags           |

---

### **9. Conceptual Summary**

LangGraph configuration transforms workflows from **hard-coded pipelines** into **flexible, policy-driven systems**:

> **Graph defines what happens.
> Configuration defines how it happens.**


### Demonstration

In [2]:
# One-cell demonstration: LangGraph configuration management in action

from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField
from langchain_classic.callbacks import StdOutCallbackHandler

# -------------------------
# 1. Define State
# -------------------------
class State(TypedDict):
    question: str
    answer: str

# -------------------------
# 2. Create configurable LLM
# -------------------------
llm = ChatOpenAI().configurable_fields(
    model_name=ConfigurableField(id="model"),
    temperature=ConfigurableField(id="temperature"),
    max_tokens=ConfigurableField(id="max_tokens")
)

# -------------------------
# 3. Define graph node
# -------------------------
def reasoning_node(state: State):
    response = llm.invoke(state["question"])
    return {"answer": response.content}

# -------------------------
# 4. Build graph
# -------------------------
builder = StateGraph(State)
builder.add_node("reason", reasoning_node)
builder.set_entry_point("reason")
builder.add_edge("reason", END)
graph = builder.compile()

# -------------------------
# 5. Run with different configurations
# -------------------------
config = {
    "configurable": {
        "model": "gpt-4o-mini",
        "temperature": 0.2,
        "max_tokens": 100
    },
    "recursion_limit": 5,
    "callbacks": [StdOutCallbackHandler()],
    "tags": ["demo", "config-management"],
    "metadata": {"env": "development"}
}

result = graph.invoke(
    {"question": "Explain overfitting in one sentence."},
    config=config
)

print("\nFinal Output:\n", result["answer"])




> Entering new LangGraph chain...


> Entering new reason chain...

> Finished chain.

> Finished chain.

Final Output:
 Overfitting occurs when a machine learning model learns the training data too well, capturing noise and outliers, which results in poor generalization to new, unseen data.
